The following script was written to export Band data from a LANDSAT8 image taken over BC, Canada.

The data will be used to train a machine learning model to predict Band 4 from LANDSAT8 using the measurements from the other bands in anoother script:

`landsat8_B4_skl_prediction.ipynb`

A smaller sample region in BC, and another in the Sahara desert will also be exported as testing tasks for the trained models.

In [1]:
from pprint import pprint       # Function for better formatted printing to console
import ee                       # Google Earth Engine
import folium                   # Visualization library for GIS data and maps
import tensorflow as tf         # Machine learning platform (includes high-level API keras as tf.keras)
import json                     # JSON file manipulation library
import time

In [2]:
ee.Initialize()                 # Necessary to read credentials from credentials.json and authenticate future commands

In [3]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
# Use Landsat 8 surface reflectance data. Image collection from Earth Engine libraries
l8sr = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Cloud masking function.
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

# The image input data is a 2018 cloud-masked median composite.
image = l8sr.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# Use folium to visualize the imagery.
mapIdDict = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
map = folium.Map(location=[45.4, -75.7])
folium.TileLayer(
    tiles=mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [4]:
# Sample the image at the points and add a random column.
sampleRegion = ee.Geometry.Rectangle([-139, 48.5, -114, 60])            # Rectangular region in BC
sample = image.sample(region=sampleRegion, scale=10000).randomColumn()  # Sample band data from the sample region at a scale of 10000m (larger pixel scale)

# Partition the sample approximately 70-30.
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

# This is list of all the properties we want to export.
featureNames = list(bands)

In [5]:
# Wait loop for Earth Engine tasks to complete. Polls for the task status the specificed number of seconds until it is no longer active
def task_wait_loop(ee_task, wait_interval):
    prev_task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
    print(prev_task_status)
    while ee_task.active():
        task_status = ee.data.getTaskStatus(ee_task.id)[0]["state"]
        if(task_status != prev_task_status):
            print(task_status)
        prev_task_status = task_status
        time.sleep(wait_interval)
    print(ee.data.getTaskStatus(ee_task.id)[0]["state"])

In [6]:
export_folder = 'hemit'

In [7]:
testingTaskCSV = ee.batch.Export.table.toDrive(
  collection=testing,
  description='Testing Export',
  folder=export_folder,
  fileFormat='csv',
  selectors=featureNames
)
testingTaskCSV.start()
task_wait_loop(testingTaskCSV, 5)

READY
RUNNING
COMPLETED


In [10]:
ee.data.getTaskStatus(testingTaskCSV.id)[0]["description"]

'Testing Export'

In [15]:
testingTaskTFRecord = ee.batch.Export.table.toDrive(
  collection=testing,
  description='Testing Export',
  folder=export_folder,
  fileFormat='TFRecord',
  selectors=featureNames
)
testingTaskTFRecord.start()
task_wait_loop(testingTaskTFRecord, 5)

READY
RUNNING
COMPLETED


In [16]:
trainingTaskCSV = ee.batch.Export.table.toDrive(
  collection=training,
  description='Training Export',
  folder=export_folder,
  fileFormat='csv',
  selectors=featureNames
)
trainingTaskCSV.start()
task_wait_loop(trainingTaskCSV, 5)

READY
RUNNING
COMPLETED


In [17]:
trainingTaskTFRecord = ee.batch.Export.table.toDrive(
  collection=training,
  description='Training Export',
  folder=export_folder,
  fileFormat='TFRecord',
  selectors=featureNames
)
trainingTaskTFRecord.start()
task_wait_loop(trainingTaskTFRecord, 5)

READY
RUNNING
COMPLETED


In [18]:
# Creating two export regions, one in BC and one in the Sahara to try to predict on using a machine learning model in another script
exportRegionBC = ee.Geometry.Rectangle([-122, 58, -121, 59])
exportRegionSahara = ee.Geometry.Rectangle([13, 24, 14, 25])

# Sampling the bands in the region at a scale of 100m, which is relatively dense sampling
exportBC = image.sample(region=exportRegionBC, scale=100)
exportSahara = image.sample(region=exportRegionSahara, scale=100)

In [19]:
BCimageTaskCSV = ee.batch.Export.table.toDrive(
  collection=exportBC,
  description='BC Image Export',
  folder=export_folder,
  fileFormat='csv',
  selectors=featureNames
)
BCimageTaskCSV.start()
task_wait_loop(BCimageTaskCSV, 5)

READY
RUNNING
COMPLETED


In [21]:
SaharaImageTaskCSV = ee.batch.Export.table.toDrive(
  collection=exportSahara,
  description='Sahara Image Export',
  folder=export_folder,
  fileFormat='csv',
  selectors=featureNames,
)
SaharaImageTaskCSV.start()
task_wait_loop(SaharaImageTaskCSV, 1)

READY
RUNNING
COMPLETED


In [23]:
# Print the list of all tasks in Earth Engine to verify that the export tasks completed successfully
for task in ee.batch.Task.list():
    print(task)

<Task EXPORT_FEATURES: Sahara Image Export (COMPLETED)>
<Task EXPORT_FEATURES: Sahara Image Export (COMPLETED)>
<Task EXPORT_FEATURES: BC Image Export (COMPLETED)>
<Task EXPORT_FEATURES: Training Export (COMPLETED)>
<Task EXPORT_FEATURES: Training Export (COMPLETED)>
<Task EXPORT_FEATURES: Testing Export (COMPLETED)>
<Task EXPORT_FEATURES: Testing Export (COMPLETED)>
<Task EXPORT_FEATURES: Testing Export (COMPLETED)>
<Task EXPORT_FEATURES: Testing Export (COMPLETED)>
<Task EXPORT_FEATURES: Testing Export (COMPLETED)>
<Task INGEST_TABLE: Ingest table: "projects/earthengine-legacy/assets/users/hemitshah/nnet2" (COMPLETED)>
<Task EXPORT_IMAGE: 20200221T170321_20200221T171237_T14SQC (COMPLETED)>
<Task EXPORT_IMAGE: 20200221T170321_20200221T171237_T14SQC (COMPLETED)>
<Task EXPORT_IMAGE: Mosaic_Export (COMPLETED)>
<Task INGEST_TABLE: Ingest table: "projects/earthengine-legacy/assets/users/hemitshah/nnet" (COMPLETED)>
<Task INGEST_TABLE: Ingest table: "projects/earthengine-legacy/assets/users